In [1]:
{-# LANGUAGE BangPatterns #-}
import AdventOfCode

import qualified Data.Vector as V
import qualified Data.Map.Strict as M

input <- dayLinesString 22

In [2]:
let len = length input `div` 2
withIndex = zip (map (\l -> zip l $ map (subtract len) [0..]) input) $ map (len+) [0,-1..]

grid = M.fromList [((x,y),c) | (l,y) <- withIndex, (c, x) <- l]
-- withIndex

In [3]:
data Direction = U | D | L | R deriving (Eq, Show)
data Turn = Lft | Rgt deriving (Eq, Show)

forward d (x,y) = case d of
    U -> (x,y+1)
    D -> (x,y-1)
    L -> (x-1,y)
    R -> (x+1,y)

turn t d = case t of
    Lft -> case d of
        U -> L
        L -> D
        D -> R
        R -> U
    Rgt -> case d of
        U -> R
        R -> D
        D -> L
        L -> U

burst gr coord direction count = case M.findWithDefault '.' coord gr of
   '.' -> let
       direction' = turn Lft direction
       gr' = M.insert coord '#' gr
       count' = count + 1
       coord' = forward direction' coord
       in (gr', coord', direction', count')
   '#' -> let
       direction' = turn Rgt direction
       gr' = M.insert coord '.' gr
       coord' = forward direction' coord
       in (gr', coord', direction', count)

loop gr coord direction count 0 = count
loop gr coord direction count n = let
    (gr', coord', direction', count') = burst gr coord direction count
    in loop gr' coord' direction' count' (n-1)

loop grid (0,0) U 0 10000

5182

In [4]:
rev d = case d of
    U -> D
    L -> R
    D -> U
    R -> L

burst' gr coord direction count = case M.findWithDefault '.' coord gr of
    '.' -> let
        direction' = turn Lft direction
        gr' = M.insert coord 'W' gr
        coord' = forward direction' coord
        in (gr', coord', direction', count)
    '#' -> let
        direction' = turn Rgt direction
        gr' = M.insert coord 'F' gr
        coord' = forward direction' coord
        in (gr', coord', direction', count)
    'W' -> let
        gr' = M.insert coord '#' gr
        coord' = forward direction coord
        in (gr', coord', direction, count+1)
    'F' -> let
        gr' = M.insert coord '.' gr
        direction' = rev direction
        coord' = forward direction' coord
        in (gr', coord', direction', count)

loop' !gr !coord !direction !count 0 = count
loop' !gr !coord !direction !count !n = let
    (gr', coord', direction', count') = burst' gr coord direction count
    in loop' gr' coord' direction' count' (n-1)

loop' grid (0,0) U 0 10000000

2512008